# Amazon Product Intelligence Data Builder

Transform raw Amazon product responses (via RapidAPI) into an AI‑ready, sentiment‑aware, query‑efficient dataset stored in MongoDB.

## What This Notebook Does
- Ingests a list of ASINs from CSV and deduplicates them.
- Fetches product metadata & reviews with controlled rate limiting and bounded retries.
- Normalizes prices and ratings into numeric fields for analytical and range queries.
- Infers product type (phone, laptop, tablet, audio, watch) from title heuristics.
- Samples and categorizes reviews (positive / negative / neutral) for sentiment‑driven chat use cases.
- Persists enriched records to MongoDB with performance‑oriented indexes (price, rating, brand, type, text search).
- Provides example MongoDB queries for ChatAI integration.

## Why It’s Useful
Optimized for powering conversational recommendation, filtering (e.g. “phones under 20k, rating > 4”), and sentiment summaries without heavy post‑processing. The schema aligns with natural language intent translation.

## High-Level Pipeline
1. Load configuration & environment variables.
2. Read ASINs from `asins.csv`.
3. Fetch product details (retry + rate limit).
4. Normalize & enrich (pricing, rating, sentiment, type detection).
5. Save to JSON and optionally clean errors externally.
6. Upsert into MongoDB and build indexes.
7. Run sample AI‑oriented queries.

## Prerequisites
- Python 3.8+
- RapidAPI key in `.env` (`RAPID_API_KEY`).
- Local or Atlas MongoDB instance.
- CSV file containing an `asin` column.

## Execution Order (Cells)
1. Install dependencies.
2. Imports & environment load.
3. ASIN reader.
4. API fetcher.
5. Normalization logic.
6. Save helpers.
7. MongoDB push & indexing.
8. Scrape runner.
9. Push cleaned JSON.
10. Sample queries for ChatAI.

Proceed to the next cell to set up dependencies.

In [7]:
%pip install -q requests retrying pandas pymongo python-dotenv tqdm

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import time
import logging
from typing import List, Dict
import requests 
import pandas as pd
from pymongo import MongoClient, UpdateOne
from dotenv import load_dotenv
from tqdm import tqdm
from retrying import retry

In [10]:
load_dotenv()

# Load from environment if available, else use defaults
RAPIDAPI_KEY = os.getenv('RAPID_API_KEY') 
RAPIDAPI_HOST = os.getenv('RAPIDAPI_HOST', 'real-time-amazon-data.p.rapidapi.com')
MONGO_URI = os.getenv('MONGODB_URI', 'mongodb://localhost:27017/') # if not then else
MONGO_DB = os.getenv('MONGO_DB', 'amazon_scrape_db')
MONGO_COLLECTION = os.getenv('MONGO_COLLECTION', 'amazon_scrape')

API_URL = 'https://real-time-amazon-data.p.rapidapi.com/product-details'

In [11]:
## Step 4: Read ASINs from a CSV file

def read_asins_from_csv(path: str) -> List[str]:
   df = pd.read_csv(path, dtype=str)
   if 'asin' not in df.columns:
      raise ValueError('CSV must contain a column named "asin".')
   asins = df['asin'].dropna().str.strip().str.upper().unique().tolist()
   return asins

In [12]:
## Step 5: Fetch data from RapidAPI

# Configure logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@retry(stop_max_attempt_number=2, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def call_api_for_asin(asin: str, country: str = 'IN') -> Dict:
	headers = {
		'X-RapidAPI-Key': RAPIDAPI_KEY,
		'X-RapidAPI-Host': RAPIDAPI_HOST
	}
	params = {
		'asin': asin,
		'country': 'IN'  # Add country parameter - 'IN' for India, 'US' for USA
	}
	
	response = requests.get(API_URL, headers=headers, params=params, timeout=15)
	if response.status_code == 200:
		return response.json()
	else:
		logger.warning(f"Error {response.status_code} for ASIN {asin}")
		response.raise_for_status()

In [13]:
## Step 6: Normalize API Response

import random
import re

def normalize_api_response(asin: str, data: Dict) -> Dict:
	# Extract the nested 'data' field from API response
	product_data = data.get('data', {}) if isinstance(data, dict) else {}
	
	# Extract all reviews (top_reviews + top_reviews_global)
	all_reviews = []
	
	# Get top reviews
	if product_data.get('top_reviews'):
		all_reviews.extend(product_data.get('top_reviews', []))
	
	# Get top reviews global
	if product_data.get('top_reviews_global'):
		all_reviews.extend(product_data.get('top_reviews_global', []))
	
	# Remove duplicates based on review_title and review_comment
	seen = set()
	unique_reviews = []
	for review in all_reviews:
		review_key = (review.get('review_title'), review.get('review_comment')[:100] if review.get('review_comment') else None)
		if review_key not in seen:
			seen.add(review_key)
			unique_reviews.append(review)
	
	# Select 10 random reviews (or fewer if less than 10 available)
	selected_reviews = random.sample(unique_reviews, min(10, len(unique_reviews))) if unique_reviews else []
	
	# Parse price to numeric value for easy querying
	def parse_price(price_str):
		"""Extract numeric price from string like '₹19,999' or '$299.99'"""
		if not price_str:
			return None
		# Remove currency symbols and commas, extract number
		numeric = re.sub(r'[^\d.]', '', str(price_str))
		try:
			return float(numeric) if numeric else None
		except:
			return None
	
	price_numeric = parse_price(product_data.get('product_price'))
	original_price_numeric = parse_price(product_data.get('product_original_price'))
	
	# Parse rating to float
	def parse_rating(rating_str):
		"""Convert rating string to float"""
		if not rating_str:
			return None
		try:
			return float(str(rating_str).replace(' out of 5 stars', '').strip())
		except:
			return None
	
	rating_numeric = parse_rating(product_data.get('product_star_rating'))
	
	# Categorize reviews as positive, negative, neutral
	positive_reviews = []
	negative_reviews = []
	neutral_reviews = []
	
	for review in selected_reviews:
		review_data = {
			'rating': review.get('review_star_rating'),
			'title': review.get('review_title'),
			'comment': review.get('review_comment'),
			'verified': review.get('review_verified_purchase'),
			'helpful_count': review.get('review_helpful_count')
		}
		
		# Parse review rating
		review_rating = parse_rating(review.get('review_star_rating'))
		
		if review_rating and review_rating >= 4:
			positive_reviews.append(review_data)
		elif review_rating and review_rating <= 2:
			negative_reviews.append(review_data)
		else:
			neutral_reviews.append(review_data)
	
	# Extract product title and create searchable keywords
	title = product_data.get('product_title', '')
	title_lower = title.lower() if title else ''
	
	# Detect product type from title
	product_type = None
	if any(keyword in title_lower for keyword in ['phone', 'mobile', 'smartphone', 'iphone', 'samsung', 'oneplus', 'redmi', 'realme']):
		product_type = 'phone'
	elif any(keyword in title_lower for keyword in ['laptop', 'notebook', 'macbook', 'chromebook']):
		product_type = 'laptop'
	elif any(keyword in title_lower for keyword in ['headphone', 'earphone', 'earbud', 'airpod']):
		product_type = 'audio'
	elif any(keyword in title_lower for keyword in ['watch', 'smartwatch']):
		product_type = 'watch'
	elif any(keyword in title_lower for keyword in ['tablet', 'ipad']):
		product_type = 'tablet'
	
	# Extract only essential fields - optimized for AI chat queries
	return {
		'asin': asin,
		'title': title,
		'brand': product_data.get('product_information', {}).get('Brand'),
		'product_type': product_type,  # For easy filtering by category
		
		# Price fields - numeric for range queries
		'price': product_data.get('product_price'),  # Original string
		'price_numeric': price_numeric,  # For queries like "under 20k"
		'original_price': product_data.get('product_original_price'),
		'original_price_numeric': original_price_numeric,
		'currency': product_data.get('currency'),
		'country': product_data.get('country'),
		
		# Rating fields - numeric for comparisons
		'rating': product_data.get('product_star_rating'),  # Original string
		'rating_numeric': rating_numeric,  # For queries like "rating > 4"
		'reviews_count': product_data.get('product_num_ratings'),
		
		# Product details
		'availability': product_data.get('product_availability'),
		'product_url': product_data.get('product_url'),
		'product_photo': product_data.get('product_photo'),
		'is_best_seller': product_data.get('is_best_seller'),
		'sales_volume': product_data.get('sales_volume'),
		'category': product_data.get('category', {}).get('name'),
		'product_description': product_data.get('product_description'),
		'customers_say': product_data.get('customers_say'),
		
		# Reviews categorized for AI queries
		'positive_reviews': positive_reviews,  # Rating >= 4
		'negative_reviews': negative_reviews,  # Rating <= 2
		'neutral_reviews': neutral_reviews,    # Rating 3
		
		# Stats for quick reference
		'total_reviews_available': len(unique_reviews),
		'positive_reviews_count': len(positive_reviews),
		'negative_reviews_count': len(negative_reviews),
		'neutral_reviews_count': len(neutral_reviews),
		
		# Metadata
		'scraped_at': None,  # Will be set during MongoDB insert
	}


In [ ]:
# ## Step 7: Save results to JSON

import json

def save_to_json(records: List[Dict], output_path: str):
    """Save scraped product data into a JSON file."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)
    logger.info(f"Saved {len(records)} records to {output_path}")

def save_full_backup(records: List[Dict], backup_path: str = 'amazon_results_full_backup.json'):
    """
    Optional: Save complete raw data as backup (includes all API response data).
    Only use this if you need to preserve the full API response for later analysis.
    """
    with open(backup_path, 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=2)
    logger.info(f" Full backup saved to {backup_path}")


In [ ]:
# ## Step 8: Push data to MongoDB with optimized indexes for AI chat queries

from datetime import datetime

def push_to_mongo(records: List[Dict]):
	client = MongoClient(MONGO_URI)
	db = client[MONGO_DB]
	collection = db[MONGO_COLLECTION]

	# Add timestamp to each record
	for r in records:
		r['scraped_at'] = datetime.utcnow()

	operations = []
	for r in records:
		operations.append(UpdateOne({'asin': r['asin']}, {'$set': r}, upsert=True))

	if operations:
		collection.bulk_write(operations)
		logger.info(f"Inserted/Updated {len(operations)} records into MongoDB.")
	
	# Create indexes for fast AI query performance
	try:
		# Index for price range queries: "show me phones under 20k"
		collection.create_index([('price_numeric', 1)])
		collection.create_index([('product_type', 1), ('price_numeric', 1)])
		
		# Index for rating queries: "show highly rated products"
		collection.create_index([('rating_numeric', -1)])
		
		# Index for brand queries: "show me Samsung phones"
		collection.create_index([('brand', 1)])
		
		# Index for product type filtering
		collection.create_index([('product_type', 1)])
		
		# Text index for natural language search on title and description
		collection.create_index([
			('title', 'text'),
			('product_description', 'text'),
			('brand', 'text')
		], name='text_search_index')
		
		# Compound index for complex queries: "phones under 20k with rating > 4"
		collection.create_index([
			('product_type', 1),
			('price_numeric', 1),
			('rating_numeric', -1)
		])
		
		logger.info("Created indexes for optimized AI chat queries")
	except Exception as e:
		logger.warning(f"Some indexes may already exist: {e}")


In [ ]:
# # Step 9A: Scrape Amazon Data and Save to JSON
# Run this cell first — it will read ASINs from a CSV, fetch product data using the API, and save all results into a JSON file.

import json
from datetime import datetime, timedelta
import threading

# --- Rate Limiter ---
class RateLimiter:
    def __init__(self, max_calls: int, period: float):
        self.max_calls = max_calls
        self.period = period
        self.lock = threading.Lock()
        self.calls = []

    def wait(self):
        with self.lock:
            now = datetime.now()
            self.calls = [t for t in self.calls if now - t < timedelta(seconds=self.period)]
            if len(self.calls) >= self.max_calls:
                sleep_time = (self.calls[0] + timedelta(seconds=self.period) - now).total_seconds()
                logger.info(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            self.calls.append(datetime.now())

rate_limiter = RateLimiter(max_calls=2, period=1.0)  # 2 requests per second to avoid 429 errors

# --- Configuration ---
MAX_ASINS_TO_SCRAPE = 300  # Set how many ASINs to scrape (None = scrape all)
input_csv = 'asins.csv'
output_json = 'amazon_results.json'

# --- Scraper Logic ---
asins = read_asins_from_csv(input_csv)

# Limit the number of ASINs if MAX_ASINS_TO_SCRAPE is set
if MAX_ASINS_TO_SCRAPE is not None:
    asins = asins[:MAX_ASINS_TO_SCRAPE]
    logger.info(f"Limiting scrape to first {len(asins)} ASINs")

results = []

for asin in tqdm(asins, desc='Scraping ASINs'):
    rate_limiter.wait()
    try:
        data = call_api_for_asin(asin)
        record = normalize_api_response(asin, data)
        results.append(record)
    except Exception as e:
        logger.error(f"Failed to fetch ASIN {asin}: {e}")
        results.append({'asin': asin, 'error': str(e)})

save_to_json(results, output_json)
logger.info(f"Scraping completed. Results saved to {output_json}.")


In [ ]:
# Step 9B: Push Scraped JSON Data to MongoDB
# Run this cell after scraping — it loads the JSON file and uploads all records to MongoDB.

json_path = 'amazon_results_cleaned.json'

if not os.path.exists(json_path):
    logger.error(f"JSON file '{json_path}' not found. Please run the previous cell first.")
else:
    with open(json_path, 'r', encoding='utf-8') as f:
        records = json.load(f)

    if not records:
        logger.warning("No records found in JSON file to push to MongoDB.")
    else:
        push_to_mongo(records)
        logger.info(f"Successfully pushed {len(records)} records from '{json_path}' to MongoDB.")


2025-11-13 16:21:37,163 - INFO - Inserted/Updated 98 records into MongoDB.
2025-11-13 16:21:37,433 - INFO - ✅ Created indexes for optimized AI chat queries
2025-11-13 16:21:37,440 - INFO - ✅ Successfully pushed 98 records from 'amazon_results_cleaned.json' to MongoDB.
2025-11-13 16:21:37,433 - INFO - ✅ Created indexes for optimized AI chat queries
2025-11-13 16:21:37,440 - INFO - ✅ Successfully pushed 98 records from 'amazon_results_cleaned.json' to MongoDB.


In [ ]:
# Check JSON file size and record count
import os

json_file = 'amazon_results.json'
if os.path.exists(json_file):
    file_size_mb = os.path.getsize(json_file) / (1024 * 1024)
    
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"📊 Data Summary:")
    print(f"   Total Records: {len(data)}")
    print(f"   File Size: {file_size_mb:.2f} MB")
    print(f"   Avg per record: {file_size_mb / len(data) * 1024:.2f} KB")
    
    # Sample record to show what's included
    if data:
        print(f"\nFields in each record:")
        print(f"   {', '.join(data[0].keys())}")
else:
    print(f"File '{json_file}' not found. Run the scraping cell first.")


📊 Data Summary:
   Total Records: 300
   File Size: 0.57 MB
   Avg per record: 1.94 KB

📦 Fields in each record:
   asin, title, brand, product_type, price, price_numeric, original_price, original_price_numeric, currency, country, rating, rating_numeric, reviews_count, availability, product_url, product_photo, is_best_seller, sales_volume, category, product_description, customers_say, positive_reviews, negative_reviews, neutral_reviews, total_reviews_available, positive_reviews_count, negative_reviews_count, neutral_reviews_count, scraped_at


In [ ]:
# Example queries for ChatAI integration

from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

# Query 1: "Show me all phones under 20k"
print("📱 Query 1: All phones under ₹20,000")
phones_under_20k = list(collection.find(
    {
        'product_type': 'phone',
        'price_numeric': {'$lte': 20000}
    },
    {
        'title': 1, 
        'brand': 1, 
        'price': 1, 
        'rating_numeric': 1,
        '_id': 0
    }
).sort('price_numeric', 1))
print(f"Found {len(phones_under_20k)} phones")
for phone in phones_under_20k[:3]:  # Show first 3
    print(f"  • {phone.get('title', 'N/A')[:60]} - {phone.get('price', 'N/A')} (Rating: {phone.get('rating_numeric', 'N/A')})")

print("\n" + "="*80 + "\n")

# Query 2: "What are positive reviews of iPhone 14"
print("⭐ Query 2: Positive reviews of iPhone 14")
iphone_reviews = collection.find_one(
    {
        '$text': {'$search': 'iPhone 14'},
        'positive_reviews_count': {'$gt': 0}
    },
    {
        'title': 1,
        'positive_reviews': 1,
        'positive_reviews_count': 1,
        '_id': 0
    }
)

if iphone_reviews:
    print(f"Product: {iphone_reviews.get('title', 'N/A')[:80]}")
    print(f"Total positive reviews: {iphone_reviews.get('positive_reviews_count', 0)}")
    print("\nPositive Reviews:")
    for i, review in enumerate(iphone_reviews.get('positive_reviews', [])[:3], 1):
        print(f"\n  Review {i}:")
        print(f"    Rating: {review.get('rating', 'N/A')}")
        print(f"    Title: {review.get('title', 'N/A')}")
        print(f"    Comment: {review.get('comment', 'N/A')[:150]}...")
else:
    print("No iPhone 14 found in database")

print("\n" + "="*80 + "\n")

# Query 3: "Show highly rated phones under 25k"
print("🏆 Query 3: Highly rated phones (rating >= 4) under ₹25,000")
top_phones = list(collection.find(
    {
        'product_type': 'phone',
        'price_numeric': {'$lte': 25000},
        'rating_numeric': {'$gte': 4.0}
    },
    {
        'title': 1,
        'brand': 1,
        'price': 1,
        'rating_numeric': 1,
        'positive_reviews_count': 1,
        '_id': 0
    }
).sort('rating_numeric', -1).limit(5))

print(f"Found {len(top_phones)} phones")
for phone in top_phones:
    print(f"  • {phone.get('title', 'N/A')[:60]}")
    print(f"    Price: {phone.get('price', 'N/A')} | Rating: {phone.get('rating_numeric', 'N/A')} | Positive Reviews: {phone.get('positive_reviews_count', 0)}")

print("\n" + "="*80 + "\n")

# Query 4: "What are negative reviews of [product]"
print("👎 Query 4: Negative reviews (example query)")
negative_example = collection.find_one(
    {
        'negative_reviews_count': {'$gt': 0}
    },
    {
        'title': 1,
        'negative_reviews': 1,
        'negative_reviews_count': 1,
        '_id': 0
    }
)

if negative_example:
    print(f"Product: {negative_example.get('title', 'N/A')[:80]}")
    print(f"Total negative reviews: {negative_example.get('negative_reviews_count', 0)}")
    for i, review in enumerate(negative_example.get('negative_reviews', [])[:2], 1):
        print(f"\n  Negative Review {i}:")
        print(f"    Rating: {review.get('rating', 'N/A')}")
        print(f"    Title: {review.get('title', 'N/A')}")
        print(f"    Comment: {review.get('comment', 'N/A')[:150]}...")

print("\n" + "="*80 + "\n")

# Query 5: "Best seller phones"
print("💎 Query 5: Best seller phones")
best_sellers = list(collection.find(
    {
        'product_type': 'phone',
        'is_best_seller': True
    },
    {
        'title': 1,
        'brand': 1,
        'price': 1,
        'rating_numeric': 1,
        '_id': 0
    }
).limit(5))

print(f"Found {len(best_sellers)} best seller phones")
for phone in best_sellers:
    print(f"  • {phone.get('title', 'N/A')[:60]} - {phone.get('price', 'N/A')}")


## ChatAI Integration Guide

### Key Features Added for ChatAI:

1. **Numeric Fields**: `price_numeric`, `rating_numeric` for range queries
2. **Product Type Detection**: Auto-categorizes products (phone, laptop, audio, etc.)
3. **Categorized Reviews**: Separate positive/negative/neutral review arrays
4. **Optimized Indexes**: Fast queries for price, rating, brand, product type
5. **Text Search**: Natural language search on title, description, brand

### Sample ChatAI Query Mappings:

| User Query | MongoDB Query |
|-----------|---------------|
| "Show me phones under 20k" | `{product_type: 'phone', price_numeric: {$lte: 20000}}` |
| "Positive reviews of iPhone 14" | `{$text: {$search: 'iPhone 14'}, positive_reviews_count: {$gt: 0}}` |
| "Highly rated laptops" | `{product_type: 'laptop', rating_numeric: {$gte: 4.5}}` |
| "Samsung phones between 15k-30k" | `{brand: 'Samsung', product_type: 'phone', price_numeric: {$gte: 15000, $lte: 30000}}` |
| "Best seller products" | `{is_best_seller: true}` |

### To integrate with your ChatAI:
1. Parse user intent to identify: product type, price range, brand, rating requirement
2. Use text search for product names (iPhone, Galaxy, etc.)
3. Filter by `positive_reviews` or `negative_reviews` arrays
4. Return results with title, price, rating, and relevant reviews
